<a href="https://colab.research.google.com/github/wiamfa/Deployment-flask-master/blob/master/huggingFace_Hubert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
model_checkpoint = "facebook/wav2vec2-base"
batch_size = 32
ravdess_emotions = ["neutral", "calm", "happy", "sad", "angry", "fearful", "disgust", "surprised"]



In [3]:
%%capture
!pip install datasets
!pip install transformers
!pip install librosa

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
%%capture
!apt install git-lfs

In [6]:
from transformers.utils import send_example_telemetry

send_example_telemetry("audio_classification_notebook", framework="pytorch")

#test HuggingFace on Hubert

In [7]:
from datasets import load_dataset, load_metric, Audio

In [8]:
dataset = load_dataset("xbgoose/ravdess",split="train")
metric = load_metric("accuracy")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1440 [00:00<?, ? examples/s]

<ipython-input-8-0b02b7a0bda2>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [9]:
dataset000=dataset.remove_columns([ 'modality', 'vocal_channel', 'emotional_intensity', 'statement', 'repetition', 'actor', 'gender'])

In [10]:
from datasets.tasks.text_classification import ClassLabel
#resampling to 16Khz
dataset00 = dataset000.cast_column("audio", Audio(sampling_rate=16000))
#Label identification
dataset0 = dataset00.cast_column("emotion", ClassLabel(num_classes=8, names=ravdess_emotions))


Casting the dataset:   0%|          | 0/1440 [00:00<?, ? examples/s]

In [11]:
dataset1=dataset0.train_test_split(test_size=0.2)

In [12]:
dataset1

DatasetDict({
    train: Dataset({
        features: ['audio', 'emotion'],
        num_rows: 1152
    })
    test: Dataset({
        features: ['audio', 'emotion'],
        num_rows: 288
    })
})

In [13]:
dataset1["test"][100]

{'audio': {'path': '03-01-01-01-02-02-10.wav',
  'array': array([ 6.56088232e-05,  1.06910767e-04,  8.00087582e-05, ...,
         -2.14805787e-05, -1.42984527e-05,  0.00000000e+00]),
  'sampling_rate': 16000},
 'emotion': 0}

In [14]:
dataset1["train"].features

{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'emotion': ClassLabel(names=['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised'], id=None)}

In [15]:
labels = dataset1["train"].features["emotion"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["7"]

'surprised'

In [16]:
import random
from IPython.display import Audio, display

for _ in range(5):
    rand_idx = random.randint(0, len(dataset1["train"])-1)
    example = dataset1["train"][rand_idx]
    audio = example["audio"]

    print(f'Label: {id2label[str(example["emotion"])]}')
    print(f'Shape: {audio["array"].shape}, sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio["array"], rate=audio["sampling_rate"]))
    print()

Label: calm
Shape: (64598,), sampling rate: 16000



Label: calm
Shape: (72072,), sampling rate: 16000



Label: sad
Shape: (57124,), sampling rate: 16000



Label: sad
Shape: (54989,), sampling rate: 16000



Label: sad
Shape: (52853,), sampling rate: 16000


In [17]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [18]:
max_duration = 2.0  # seconds

In [19]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
    )
    return inputs

In [20]:
preprocess_function(dataset1['train'][:5])

{'input_values': [array([-2.1241297e-04, -2.1393914e-04, -2.1371967e-04, ...,
        5.9428781e-02,  2.5171298e-01,  3.8278928e-01], dtype=float32), array([-6.5220590e-04, -6.0319289e-04, -4.7948555e-04, ...,
       -2.8718686e+00, -4.2363887e+00, -6.6938910e+00], dtype=float32), array([0.00030023, 0.00029897, 0.00030851, ..., 0.0332164 , 0.06599698,
       0.12602152], dtype=float32), array([-0.00074413, -0.00074413, -0.00074413, ..., -0.61192465,
       -0.49455315, -0.5529103 ], dtype=float32), array([0.01647174, 0.02510256, 0.02295839, ..., 2.3130224 , 2.3364267 ,
       2.104011  ], dtype=float32)]}

In [21]:
encoded_dataset = dataset1.map(preprocess_function, remove_columns=["audio"], batched=True)
encoded_dataset

Map:   0%|          | 0/1152 [00:00<?, ? examples/s]

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['emotion', 'input_values'],
        num_rows: 1152
    })
    test: Dataset({
        features: ['emotion', 'input_values'],
        num_rows: 288
    })
})

In [22]:
encoded_dataset["train"][0]["emotion"]

4

In [23]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'projector.weight', 'classifier.weight', 'projector.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.9 MB/s eta 0:00:00


In [ ]:
!accelerate config


2023-08-17 14:16:59.789591: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
In which compute environment are you running?
Please input a choice index (starting from 0), and press enter
 ➔  This machine
    AWS (Amazon SageMaker)


In [ ]:
!accelerate env

In [ ]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-ravdess",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    gradient_checkpointing=True,


)

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
encoded_dataset["train"]

In [ ]:
encoded_dataset1= encoded_dataset.rename_column("emotion", "labels")

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset1["train"],
    eval_dataset=encoded_dataset1["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.push_to_hub()